The script assumes that the training/test data consists of a list of tables. Each table corresponds to a week. The rows of table show the days in the week, and the columns are the periods in the day.

The paper assumes 30-minute periods, whihch leads to 48 periods per day. Here I create a toy dataset with 2 weeks and 3 periods per day:

In [ ]:
d1 = matrix(c(1:(3*7)*3), nrow=7, ncol=3)
d2 = matrix(c(1:(3*7)*5), nrow=7, ncol=3)
d = list(d1, d2)

`table_pr` gives labels to the columns and rows of a table. The rows are named by the days of week (*Monday*, ..., *Sunday*), and the columns are named *Period1*, ... *Periodx*.

In [ ]:
table_pr = function(m) {  
  rownames(m) <- weekdays(as.Date(4,"1970-01-01",tz="GMT")+0:6)
  colnames(m)<-colnames(m, do.NULL = FALSE, prefix = "Period")
  m=as.table(m)
  return(m)
}

In [ ]:
table_pr(d[[1]])

The function `calculsum` computes a sum over multiple weeks. It takes the day as argument, and iterates over the periods in the day.

In [ ]:
calculsum = function(z,n,liste) {                   
  # I changed the number of periods from 48 to 3  
  sapply(1:3,function(y) sum(sapply(1:n, function(x)liste[[x]][z,y])))
}

By applying `calculsum` across the days, we will get a single matrix that sums the matrices in training data:

In [ ]:
sum_past = lapply(1:7, function(x) calculsum(x, 2, d))
sum_past

Do type conversion and put labels using `table_pr`:

In [ ]:
# I changed the number of periods from 48 to 3
sum_past <- table_pr(matrix(unlist(sum_past), ncol = 3, byrow = TRUE))
sum_past

To compute the average, create a matrix populated with number of weeks. The divide the sum matrix by this one:

In [ ]:
# 48 to 3
nbr_week=table_pr(matrix(2, ncol = 3, nrow = 7))
p_1=round(sum_past/nbr_week)
p_1

Let's create a toy dataset:

In [ ]:
t1 = matrix(round(c(1:(3*7)*1.5)), nrow=7, ncol=3)
t2 = matrix(round(c(1:(3*7)*2.5)), nrow=7, ncol=3)
t3 = matrix(round(c(1:(3*7)*3.5)), nrow=7, ncol=3)
t = list(t1, t2, t3)

Now the script does something weird. For each period in test, it replace that period with a period in the train, and recomputes the sum. 

In [ ]:
# I did rename things a bit
n1 = 3
sum_past_list=lapply(1:(n1-1),function(x) sum_past-d[[x]]+t[[x]])

In [ ]:
sum_past_list

Then the poisson list is computed. It is a modification of the averages that we had already computed, by adding replacing the past week with a week in the sum and recomputing the average.

In [ ]:
poisson_list=lapply(1:n1,function(x) if(x==1){p_1}else{round(sum_past_list[[x-1]]/nbr_week)})

In [ ]:
poisson_list

Finally, the poisson rates themselves are used as predictions:

In [ ]:
pred_p=unlist(lapply(1:n1, function(x) as.vector(t(poisson_list[[x]]))))
pred_p